In [1]:
import pandas as pd

def preprocessing(filepath, year):
    
    data = pd.read_csv(filepath)
    # Drop all rows with 'total' in the column 'Sesso'
    indices_to_drop = list(data[data['Sesso'] == 'Totale'].index)
    data.drop(indices_to_drop, inplace=True)
    # Adjust the column 'Età' before using pd.cat
    data['Età'] = data['Età'].replace('100 e oltre', 100)
    data['Età'] = data['Età'].astype('int64')
    # Use pd.cut to make age groups
    bins = [-1, 30, 65, 150]
    group = ['Young', 'Middle-age', 'Old']
    data['age_group'] = pd.cut(data['Età'], bins=bins, labels=group, right=False)
    # Drop 'Età', don't need it
    data.drop(['Età', 'Codice comune'], axis=1, inplace=True)
    
    # Just to carry the 'Codice comune'
    # temp = data[['Comune', 'Codice comune']]
    # temp = temp.groupby(by=['Comune', 'Codice comune'])
    # temp = temp.count()
    # temp = temp.reset_index()
    
    # Use group by to sum by age group
    data = data.groupby(by=['age_group', 'Comune', 'Sesso']).sum()
    data = data.reset_index()
    # Set the year
    data['Year'] = year
    # Merge with temp
    # data = data.merge(temp, left_on='Comune', right_on='Comune')
    
    return data

In [2]:
path_list = ['DEMO_2013.csv', 'DEMO_2014.csv', 'DEMO_2015.csv', 'DEMO_2016.csv', 'DEMO_2017.csv', 'DEMO_2018.csv', 'DEMO_2019.csv']
years = ['2013', '2014', '2015', '2016', '2017', '2018', '2019']
final_demo_dataset = pd.DataFrame()

for path, year in zip(path_list, years):
    data = preprocessing(path, year)
    final_demo_dataset = pd.concat([final_demo_dataset, data], axis=0)

In [3]:
final_demo_dataset

,age_group,Comune,Sesso,Popolazione,Year
0,Young,Abbiategrasso,Femmine,4423,2013
1,Young,Abbiategrasso,Maschi,4658,2013
2,Young,Albairate,Femmine,682,2013
3,Young,Albairate,Maschi,730,2013
4,Young,Arconate,Femmine,955,2013
...,...,...,...,...,...
793,Old,Vittuone,Maschi,830,2019
794,Old,Vizzolo Predabissi,Femmine,519,2019
795,Old,Vizzolo Predabissi,Maschi,451,2019
796,Old,Zibido San Giacomo,Femmine,580,2019


In [4]:
# final_demo_dataset.to_csv('Demographics.csv', index=False)

In [5]:
final_demo_dataset = final_demo_dataset[final_demo_dataset['Comune'] != 'San Colombano al Lambro']
final_demo_dataset.shape

(5544, 5)

In [6]:
final_demo_dataset.shape

(5544, 5)

In [7]:
perc_anziani = final_demo_dataset[final_demo_dataset['age_group'] == 'Old'].groupby(by=['Comune',
        'Year']).sum() / final_demo_dataset.groupby(by=['Comune', 'Year']).sum()

perc_anziani = perc_anziani.reset_index()
perc_anziani

,Comune,Year,Popolazione
0,Abbiategrasso,2013,0.213375
1,Abbiategrasso,2014,0.214651
2,Abbiategrasso,2015,0.217792
3,Abbiategrasso,2016,0.218834
4,Abbiategrasso,2017,0.221361
...,...,...,...
919,Zibido San Giacomo,2015,0.150776
920,Zibido San Giacomo,2016,0.156648
921,Zibido San Giacomo,2017,0.159442
922,Zibido San Giacomo,2018,0.162374


In [8]:
perc_anziani = perc_anziani[perc_anziani['Comune'] != 'Milano']
perc_anziani

,Comune,Year,Popolazione
0,Abbiategrasso,2013,0.213375
1,Abbiategrasso,2014,0.214651
2,Abbiategrasso,2015,0.217792
3,Abbiategrasso,2016,0.218834
4,Abbiategrasso,2017,0.221361
...,...,...,...
919,Zibido San Giacomo,2015,0.150776
920,Zibido San Giacomo,2016,0.156648
921,Zibido San Giacomo,2017,0.159442
922,Zibido San Giacomo,2018,0.162374


In [9]:
# Per i 9 municipi di Milano sono disponibili solo i dati del 2020 nel seguente pdf
# (https://www.comune.milano.it/documents/20126/2313917/MILANO+QUARTIERI+2020.pdf)

add_instances = pd.DataFrame([['Centro Storico', 2013, 0.226],
                              ['Centro Storico', 2014, 0.226],
                              ['Centro Storico', 2015, 0.226],
                              ['Centro Storico', 2016, 0.226],
                              ['Stazione Centrale Bicocca', 2013, 0.196],
                              ['Stazione Centrale Bicocca', 2014, 0.196],
                              ['Stazione Centrale Bicocca', 2015, 0.196],
                              ['Stazione Centrale Bicocca', 2016, 0.196],
                              ['Città Studi Lambrate', 2013, 0.235],
                              ['Città Studi Lambrate', 2014, 0.235],
                              ['Città Studi Lambrate', 2015, 0.235],
                              ['Città Studi Lambrate', 2016, 0.235],
                              ['Vittoria Forlanini', 2013, 0.225],
                              ['Vittoria Forlanini', 2014, 0.225],
                              ['Vittoria Forlanini', 2015, 0.225],
                              ['Vittoria Forlanini', 2016, 0.225],
                              ['Vigentino Chiaravalle Gratosoglio', 2013, 0.226],
                              ['Vigentino Chiaravalle Gratosoglio', 2014, 0.226],
                              ['Vigentino Chiaravalle Gratosoglio', 2015, 0.226],
                              ['Vigentino Chiaravalle Gratosoglio', 2016, 0.226],
                              ['Barona Lorenteggio', 2013, 0.249],
                              ['Barona Lorenteggio', 2014, 0.249],
                              ['Barona Lorenteggio', 2015, 0.249],
                              ['Barona Lorenteggio', 2016, 0.249],
                              ['San Siro Baggio Trenno', 2013, 0.239],
                              ['San Siro Baggio Trenno', 2014, 0.239],
                              ['San Siro Baggio Trenno', 2015, 0.239],
                              ['San Siro Baggio Trenno', 2016, 0.239],
                              ['Fiera Gallaratese Quarto Oggiaro', 2013, 0.236],
                              ['Fiera Gallaratese Quarto Oggiaro', 2014, 0.236],
                              ['Fiera Gallaratese Quarto Oggiaro', 2015, 0.236],
                              ['Fiera Gallaratese Quarto Oggiaro', 2016, 0.236],
                              ['Stazione Garibaldi Niguarda', 2013, 0.210],
                              ['Stazione Garibaldi Niguarda', 2014, 0.210],
                              ['Stazione Garibaldi Niguarda', 2015, 0.210],
                              ['Stazione Garibaldi Niguarda', 2016, 0.210]], columns=['Comune', 'Year', 'Popolazione'])

In [10]:
# Remove years out of our interest and append new instances

perc_anziani = perc_anziani[perc_anziani['Year'].isin(['2013', '2014', '2015', '2016'])]
perc_anziani = pd.concat([perc_anziani, add_instances], ignore_index=True)

In [11]:
perc_anziani.shape

(560, 3)

In [12]:
perc_anziani.to_csv('Perc_old_people.csv', index=False)